In [10]:
import os, sys, json, time
from pathlib import Path
import numpy as np
import pandas as pd

ROOT = os.path.abspath("..")
if ROOT not in sys.path:
    sys.path.insert(0, ROOT)

with open(os.path.join(ROOT, "config.json"), "r") as f:
    C = json.load(f)

TICKER   = C["ticker"]; START = C["start"]; END = C["end"]; INTERVAL = C["interval"]
HORIZON  = int(C["horizon"]); LOOKBACK = int(C["lookback"])
BATCH    = int(C["batch"]);   EPOCHS   = int(C["epochs"])
SEED     = int(C.get("seed", 42))

RESULTS_DIR = Path(C.get("results_dir", "../results"))
RESULTS_DIR.mkdir(parents=True, exist_ok=True)
RUN_DIR   = RESULTS_DIR / time.strftime("%Y-%m-%d_%H-%M-%S_lstm")
RUN_DIR.mkdir(parents=True, exist_ok=True)   # <- hinzufügen
print("RUN_DIR:", RUN_DIR)                   # optional

TRAIN_CSV = f"../data/{TICKER}_{INTERVAL}_{START}_{END}_cls_h{HORIZON}.csv"

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix
import joblib

np.random.seed(SEED)
tf.random.set_seed(SEED)


RUN_DIR: ..\results\2025-10-02_22-01-04_lstm


In [11]:
df = pd.read_csv(TRAIN_CSV, index_col=0, parse_dates=[0]).sort_index()

exp = {"open","high","low","close","volume","logret_1d","target"}
missing = exp - set(df.columns)
assert not missing, f"Fehlende Spalten: {missing}"
assert not df.index.has_duplicates
assert (df["close"] > 0).all()
assert df.notna().all().all()


In [12]:
# === 3) Features / Ziel wählen ===
FEATURES = ["logret_1d"]    # Start einfach; später gerne mehr Indikatoren
TARGET   = "target"

X = df[FEATURES].copy()
y = df[TARGET].astype(int).copy()

In [13]:
# === 4) Chronologische Splits (70/15/15) ===
n = len(df)
n_train = int(n * 0.70)
n_val   = int(n * 0.15)
n_test  = n - n_train - n_val

train_idx = slice(0, n_train)
val_idx   = slice(n_train, n_train + n_val)
test_idx  = slice(n_train + n_val, n)

X_train, y_train = X.iloc[train_idx], y.iloc[train_idx]
X_val,   y_val   = X.iloc[val_idx],   y.iloc[val_idx]
X_test,  y_test  = X.iloc[test_idx],  y.iloc[test_idx]

print(f"Split sizes → train {len(X_train)}, val {len(X_val)}, test {len(X_test)}")

Split sizes → train 2403, val 515, test 516


In [14]:
# === 5) Scaler nur auf TRAIN fitten ===
scaler = StandardScaler(with_mean=True, with_std=True)
X_train_s = pd.DataFrame(scaler.fit_transform(X_train), index=X_train.index, columns=FEATURES)
X_val_s   = pd.DataFrame(scaler.transform(X_val),       index=X_val.index,   columns=FEATURES)
X_test_s  = pd.DataFrame(scaler.transform(X_test),      index=X_test.index,  columns=FEATURES)

# Scaler speichern (für spätere Runs/Inference)
import joblib, io
joblib.dump(scaler, RUN_DIR / "scaler.joblib")

['..\\results\\2025-10-02_22-01-04_lstm\\scaler.joblib']

In [15]:
# === 6) Windowing: Sequenzen der Länge LOOKBACK → Label am Endzeitpunkt ===
def make_windows(X_df: pd.DataFrame, y_ser: pd.Series, lookback: int):
    X_values = X_df.values.astype(np.float32)
    y_values = y_ser.values.astype(np.int32)
    n = len(X_df)
    xs, ys = [], []
    for i in range(lookback-1, n):
        xs.append(X_values[i - lookback + 1 : i + 1])  # inkl. i
        ys.append(y_values[i])                          # Label für Zeitpunkt i (Up/Down für i->i+H)
    return np.stack(xs, axis=0), np.array(ys)

Xtr_win, ytr = make_windows(X_train_s, y_train, LOOKBACK)
Xva_win, yva = make_windows(X_val_s,   y_val,   LOOKBACK)
Xte_win, yte = make_windows(X_test_s,  y_test,  LOOKBACK)

print("Shapes:",
      "\n  train:", Xtr_win.shape, ytr.shape,
      "\n  val  :", Xva_win.shape, yva.shape,
      "\n  test :", Xte_win.shape, yte.shape)

Shapes: 
  train: (2344, 60, 1) (2344,) 
  val  : (456, 60, 1) (456,) 
  test : (457, 60, 1) (457,)


In [16]:
# === 7) tf.data Pipelines ===
def to_ds(X, y, batch, shuffle):
    ds = tf.data.Dataset.from_tensor_slices((X, y))
    if shuffle:
        ds = ds.shuffle(buffer_size=len(X), seed=SEED, reshuffle_each_iteration=True)
    return ds.batch(batch).prefetch(tf.data.AUTOTUNE)

ds_train = to_ds(Xtr_win, ytr, BATCH, shuffle=True)
ds_val   = to_ds(Xva_win, yva, BATCH, shuffle=False)
ds_test  = to_ds(Xte_win, yte, BATCH, shuffle=False)

In [17]:
# === 8) Modell definieren ===
n_features = Xtr_win.shape[-1]

model = keras.Sequential([
    layers.Input(shape=(LOOKBACK, n_features)),
    layers.LSTM(64, return_sequences=True),
    layers.Dropout(0.2),
    layers.LSTM(32),
    layers.Dense(16, activation="relu"),
    layers.Dense(1, activation="sigmoid"),
])

model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=1e-3),
    loss=keras.losses.BinaryCrossentropy(),
    metrics=[
        keras.metrics.BinaryAccuracy(name="acc"),
        keras.metrics.AUC(name="auc"),
        keras.metrics.Precision(name="prec"),
        keras.metrics.Recall(name="rec"),
    ],
)

model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ lstm_2 (LSTM)                        │ (None, 60, 64)              │          16,896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 60, 64)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_3 (LSTM)                        │ (None, 32)                  │          12,416 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 16)                  │             528 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 1)                   │              17 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 29,857 (116.63 KB)

 Trainable params: 29,857 (116.63 KB)

 Non-trainable params: 0 (0.00 B)

In [18]:
# === 9) Callbacks ===
ckpt_path = RUN_DIR / "best.keras"
callbacks = [
    keras.callbacks.ModelCheckpoint(
        filepath=str(ckpt_path),
        monitor="val_auc",
        mode="max",
        save_best_only=True,
        save_weights_only=False,
        verbose=1,
    ),
    keras.callbacks.EarlyStopping(
        monitor="val_auc", mode="max", patience=10, restore_best_weights=True
    ),
    keras.callbacks.ReduceLROnPlateau(
        monitor="val_auc", mode="max", factor=0.5, patience=5, verbose=1
    ),
]

In [19]:
# === 10) Train ===
history = model.fit(
    ds_train,
    validation_data=ds_val,
    epochs=EPOCHS,
    callbacks=callbacks,
    verbose=1,
)

# Trainingskurve speichern
pd.DataFrame(history.history).to_csv(RUN_DIR / "history.csv", index=False)

Epoch 1/100
36/37 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - acc: 0.4998 - auc: 0.4799 - loss: 0.6935 - prec: 0.5180 - rec: 0.5683
Epoch 1: val_auc improved from None to 0.48470, saving model to ..\results\2025-10-02_22-01-04_lstm\best.keras
37/37 ━━━━━━━━━━━━━━━━━━━━ 2s 21ms/step - acc: 0.5107 - auc: 0.4910 - loss: 0.6929 - prec: 0.5205 - rec: 0.8280 - val_acc: 0.5088 - val_auc: 0.4847 - val_loss: 0.6940 - val_prec: 0.5088 - val_rec: 1.0000 - learning_rate: 0.0010
Epoch 2/100
33/37 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - acc: 0.5069 - auc: 0.5235 - loss: 0.6930 - prec: 0.5059 - rec: 0.9767
Epoch 2: val_auc did not improve from 0.48470
37/37 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - acc: 0.5205 - auc: 0.5190 - loss: 0.6920 - prec: 0.5236 - rec: 0.9324 - val_acc: 0.4978 - val_auc: 0.4786 - val_loss: 0.6967 - val_prec: 0.5034 - val_rec: 0.9483 - learning_rate: 0.0010
Epoch 3/100
36/37 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - acc: 0.5395 - auc: 0.5004 - loss: 0.6906 - prec: 0.5415 - rec: 0.9815
Epoch 3: val_au

In [20]:
# === 11) Evaluate & Berichte ===
# Best Weights sind dank EarlyStopping bereits geladen
test_metrics = model.evaluate(ds_test, return_dict=True, verbose=0)
print("Test metrics:", json.dumps(test_metrics, indent=2))

# Schwellenwert 0.5 (später kalibrierbar)
y_proba = model.predict(ds_test, verbose=0).ravel()
y_pred  = (y_proba >= 0.5).astype(int)

print("\nConfusion matrix (test):\n", confusion_matrix(yte, y_pred))
print("\nClassification report (test):\n", classification_report(yte, y_pred, digits=3))

Test metrics: {
  "acc": 0.5339168310165405,
  "auc": 0.513331413269043,
  "loss": 0.6904076337814331,
  "prec": 0.5379464030265808,
  "rec": 0.9757084846496582
}

Confusion matrix (test):
 [[  3 207]
 [  6 241]]

Classification report (test):
               precision    recall  f1-score   support

           0      0.333     0.014     0.027       210
           1      0.538     0.976     0.694       247

    accuracy                          0.534       457
   macro avg      0.436     0.495     0.360       457
weighted avg      0.444     0.534     0.387       457



In [21]:
# === 12) Artefakte sichern ===
# Keras-Format (SavedModel) + Gewichte
model.save(RUN_DIR / "model.keras")
np.save(RUN_DIR / "y_test.npy", yte)
np.save(RUN_DIR / "y_proba.npy", y_proba)
with open(RUN_DIR / "config.json", "w") as f:
    json.dump({
        "ticker": TICKER, "start": START, "end": END, "interval": INTERVAL,
        "horizon": HORIZON, "lookback": LOOKBACK, "features": FEATURES,
        "scaler": "StandardScaler", "seed": SEED, "batch": BATCH, "epochs": EPOCHS
    }, f, indent=2)
print(f"\nArtefakte gespeichert in: {RUN_DIR}")


Artefakte gespeichert in: ..\results\2025-10-02_22-01-04_lstm
